In [1]:
import pandas as pd
import numpy as np
from numpy import argmax
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import librosa.display
import IPython.display
import random
import warnings
import os
from PIL import Image
import pathlib
import csv
# sklearn Preprocessing
from sklearn.model_selection import train_test_split
#Keras
import keras
import warnings
warnings.filterwarnings('ignore')
from keras import layers
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.models import Sequential
from keras.optimizers import SGD



Collab için drive işlemleri

In [2]:
#drive tanımlaması yapılmak zorunda
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [3]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

 Her ses işlenip spectrogramları png olarak kaydediyor.

In [ ]:

words=['down', 'eat', 'sleep', 'up']

current_path = "drive/My Drive/training_sets/"
 
for i in range (len(words)):
  
  pathlib.Path(f'img_data2/{words[i]}').mkdir(parents=True, exist_ok=True) #for images                           
  for j in range (10):
    path = current_path +  words[i] + "-" + str(j+1) + ".wav"  
    y, sr = librosa.load(path, mono=True, duration=5)
    print(y.shape)
    plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, sides='default', mode='default', scale='dB');
    print(y.shape)
    plt.axis('off');
    plt.savefig((f'img_data2/{words[i]}/{words[i]}-{j+1}.png'))    #Kaydet
    plt.clf()

---------------------- Another person same word check --------------------------(Optional) ->LOW LEARNİNG ACCURACY 

In [58]:
#Farklı seste aynı kelimeler analizi icin 
current_path = "drive/My Drive/test_/"
for i in range (len(words)):
  pathlib.Path(f'test_png/{words[i]}').mkdir(parents=True, exist_ok=True) #for images                           
  for j in range (10):
    path = current_path +  words[i] + "-" + str(j+1) + ".wav"
    y, sr = librosa.load(path, mono=True, duration=5)
    plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, sides='default', mode='default', scale='dB');
    plt.axis('off');
    plt.savefig((f'test_png/{words[i]}/{words[i]}-{j+1}.png'))    #Kaydet
    plt.clf()


<Figure size 432x288 with 0 Axes>

In [5]:
!pip install split-folders

In [6]:
#For  dataset
import splitfolders 

splitfolders.ratio('./img_data2/', output="./data2", seed=1337, ratio=(.8, .2)) # default values
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator( #(X_train , y_train return)
        rescale=1./255,       # all our pixel values are in range (0,1)
        shear_range=0.2,      #to apply some random tranfromations
        zoom_range=0.2,       
        horizontal_flip=True) 

test_datagen = ImageDataGenerator( #(X_test , y_test return)
        rescale=1./255,       # all our pixel values are in range (0,1)
        shear_range=0.2,      #to apply some random tranfromations
        zoom_range=0.2,      
        horizontal_flip=True) 

Copying files: 40 files [00:00, 5187.92 files/s]


In [21]:
#data generators

#class-mode -> yapılacak sınıflandırmaya göre uygun olanı seçilmelidir. (binary - categorical)     
training_set = train_datagen.flow_from_directory( 
    './data2/train',
    target_size=(64, 64),  ## input size belirlendi.
    batch_size=1,
    class_mode='binary',
    shuffle = False)

test_set = test_datagen.flow_from_directory(
    './data2/val',   # 'test_png'-> './data2/val' for another people same words
    target_size=(64, 64),    ## input size belirlendi.
    batch_size=1,
    class_mode='binary',
    shuffle = False )



Found 32 images belonging to 4 classes.
Found 8 images belonging to 4 classes.


In [44]:
model = Sequential()
input_shape=(64, 64, 3)                                              #1st hidden layer
model.add(Conv2D(32, (3, 3), strides=(2, 2), input_shape=input_shape))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))                                         #2nd hidden layer
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))                                         #3rd hidden layer
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))                                         #Flatten
model.add(Flatten())
model.add(Dropout(rate=0.5))                                          #Add fully connected layer.
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))                                          #Output layer
model.add(Dense(10))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 31, 31, 32)        896       
_________________________________________________________________
average_pooling2d_28 (Averag (None, 15, 15, 32)        0         
_________________________________________________________________
activation_44 (Activation)   (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 15, 15, 64)        18496     
_________________________________________________________________
average_pooling2d_29 (Averag (None, 7, 7, 64)          0         
_________________________________________________________________
activation_45 (Activation)   (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 7, 7, 64)         

In [45]:
epochs = 200
batch_size = 8
learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.9
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
model.compile(optimizer="sgd",  loss ="sparse_categorical_crossentropy", metrics=['accuracy']) # sgd, adam dan daha basarili 

In [46]:
model.fit_generator(
        training_set,
        steps_per_epoch= len(training_set), 
        epochs=200,
        validation_data=test_set,
        validation_steps= len(test_set), 
         )

Epoch 1/200
32/32 [==============================] - 1s 12ms/step - loss: 2.2847 - accuracy: 0.1345 - val_loss: 2.0669 - val_accuracy: 0.2500
Epoch 2/200
32/32 [==============================] - 0s 8ms/step - loss: 1.9650 - accuracy: 0.2844 - val_loss: 1.7493 - val_accuracy: 0.2500
Epoch 3/200
32/32 [==============================] - 0s 9ms/step - loss: 1.7255 - accuracy: 0.2976 - val_loss: 1.7135 - val_accuracy: 0.2500
Epoch 4/200
32/32 [==============================] - 0s 8ms/step - loss: 1.7879 - accuracy: 0.1883 - val_loss: 1.5897 - val_accuracy: 0.2500
Epoch 5/200
32/32 [==============================] - 0s 8ms/step - loss: 1.7654 - accuracy: 0.3264 - val_loss: 1.5145 - val_accuracy: 0.2500
Epoch 6/200
32/32 [==============================] - 0s 8ms/step - loss: 1.6811 - accuracy: 0.3188 - val_loss: 1.5083 - val_accuracy: 0.2500
Epoch 7/200
32/32 [==============================] - 0s 8ms/step - loss: 1.6905 - accuracy: 0.2502 - val_loss: 1.5207 - val_accuracy: 0.2500
Epoch 8/200


In [47]:
#Model Evaluation
model.evaluate_generator(generator=test_set, steps=200)

[0.3081400394439697, 0.875]

In [48]:
#test veri setini değiştiriyoruz. Ama önce sıfırlanması gerekiyor
test_set.reset()
pred = model.predict_generator(test_set, verbose=1) #steps= len(test_set)

8/8 [==============================] - 0s 5ms/step


In [49]:
predicted_class_indices=np.argmax(pred,axis=1)
#print(predicted_class_indices)

labels = (training_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
predictions = predictions[:600]
filenames=test_set.filenames
#print(filenames)

In [50]:
print(len(filenames), len(predictions))


8 8


Karşılaştırmalar için tablo oluşturuldu.

In [51]:
import csv
results=pd.DataFrame({"Filename" :filenames,
                      "Predictions":predictions})

    
results.to_csv("prediction_results.csv",index=False)

In [53]:
from keras.models import model_from_json

#%% model save .json

model_json = model.to_json()

with open("drive/My Drive/voice_analysis.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("drive/My Drive/voice_analysis.h5")


Eğitilmiş Ağ kaydediliyor ve yükleniyor.


In [54]:
json_file = open('drive/My Drive/voice_analysis.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("drive/My Drive/voice_analysis.h5")


Farklı kişi sesleriyle eğitilmiş ağ test ediliyor. (Doğruluk değerinin azalması beklenir. Ses tanıma kişiye özeldir.)

In [59]:
test_set = test_datagen.flow_from_directory(
    'test_png',   # 'test_png'-> './data2/val' for another people same words
    target_size=(64, 64),    ## input size belirlendi.
    batch_size=1,
    class_mode='binary',
    shuffle = False )

Found 40 images belonging to 4 classes.


In [107]:
#Egitime yuklenen ag ile deneme
from keras.models import model_from_json

loaded_model_predict = loaded_model.predict(test_set)
model = model_from_json(loaded_model_json)
#load weights into new model
model.load_weights("drive/My Drive/voice_analysis.h5")
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
                                                

y_final_oneHotEncoded= model.predict_classes(test_set, batch_size=1, verbose=0)  
predictNumber = y_final_oneHotEncoded[0]
words=['down', 'eat', 'sleep', 'up']

temp=0
for i in range (len(y_final_oneHotEncoded)):
  original = test_set.filenames[i]
  predict =  words[y_final_oneHotEncoded[i]]

  if i<10:
    orig = "down"
  elif i<20:
    orig= "eat"
  elif i<30:
    orig= "sleep"
  else:
    orig= "up"

  print("original:" , orig ," ", original )
  print("prediction:" , predict)

  if orig == predict :
    temp = temp+1
    print("true")
  else:
    print("wrong")


 
print("acc:", temp / 40)

original: down   down/down-1.png
prediction: up
wrong
original: down   down/down-10.png
prediction: sleep
wrong
original: down   down/down-2.png
prediction: sleep
wrong
original: down   down/down-3.png
prediction: down
true
original: down   down/down-4.png
prediction: eat
wrong
original: down   down/down-5.png
prediction: up
wrong
original: down   down/down-6.png
prediction: up
wrong
original: down   down/down-7.png
prediction: sleep
wrong
original: down   down/down-8.png
prediction: sleep
wrong
original: down   down/down-9.png
prediction: sleep
wrong
original: eat   eat/eat-1.png
prediction: eat
true
original: eat   eat/eat-10.png
prediction: eat
true
original: eat   eat/eat-2.png
prediction: sleep
wrong
original: eat   eat/eat-3.png
prediction: eat
true
original: eat   eat/eat-4.png
prediction: eat
true
original: eat   eat/eat-5.png
prediction: down
wrong
original: eat   eat/eat-6.png
prediction: eat
true
original: eat   eat/eat-7.png
prediction: eat
true
original: eat   eat/eat-8.pn